In [ ]:
using LinearAlgebra
using Optim
using Plots

# Definición variables valores_iniciales
function variable_inicial(valores_iniciales, parametros)
    _, N, _, _, _, _ = parametros
    variable_inicial = zeros(N, 3)
    variable_inicial[1, :] = valores_iniciales
    return vec(variable_inicial')
end

# Definición del kernel en este caso RBF
function kernel(x, x_gorro, l, i, parametros)
    _, _, _, _, sigma, _ = parametros
    return exp(-(1/(2*sigma^2)) * ((x[l, 1]-x_gorro[i, 1])^2 + (x[l, 2]-x_gorro[i, 2])^2))
end

# Definición de la función objetivo
function funcion_objetivo(vars, kernel, valores_iniciales, parametros)
    T, N, vf, lambda_value, _, gamma = parametros
    # Extraer las variables
    x = reshape(vars[1:2*N], 2, N)'
    alpha = vars[2*N+1:end]
    # Imprimir el valor de la función objetivo
    println("valor", (x[N, 2]-vf)^2 + lambda_value*sum(alpha.^2))
    return (x[N, 2]-vf)^2 + lambda_value*sum(alpha.^2)
end

T = 10
N = 30
vf = 100
lambda_value = 0.0001
sigma = 1
gamma = 1

# Parámetros
parametros = [T, N, vf, lambda_value, sigma, gamma]

# x0, v0, alpha0
valores_iniciales = [80, 80, 0.1]

variable_inicial = variable_inicial(valores_iniciales, parametros)

# Definir la restricción
function constraint(vars)
    x = reshape(vars[1:2*N], 2, N)'
    alpha = vars[2*N+1:end]
    constraints = []
    push!(constraints, x[1, 1] - valores_iniciales[1])
    push!(constraints, x[1, 2] - valores_iniciales[2])
    for i in 1:N-1
        push!(constraints, x[i+1, 1] - x[i, 1] + (T/N)*(x[i, 2]))
        push!(constraints, x[i+1, 2] - x[i, 2] + (T/N)*(-gamma*x[i, 1] + x[i, 2] + sum(alpha[l]*kernel(x, x, l, i, parametros) for l in 1:N)))
    end
    return vcat(constraints...)
end

# Definir las restricciones en el formato requerido por Optim
constraints = NonlinearConstraints(constraint, zeros(2*N))

# Minimizar la función objetivo con restricciones
result = optimize(vars -> funcion_objetivo(vars, kernel, valores_iniciales, parametros), variable_inicial, constraints, method=:ipopt)

# Ejecución de la optimización
println("primera optimización", result)

# Graficar las soluciones
function plot(result, valores_iniciales, parametros)
    T, N, vf, lambda_value, _, gamma = parametros

    # Extraer las soluciones
    x_sol = reshape(result.minimizer[1:2*N], 2, N)'
    alpha_sol = result.minimizer[2*N+1:end]

    # creación de x_gorro
    x_gorro = x_sol

    # Definición del control
    u_opt = zeros(N)
    for i in 1:N
        u_opt[i] = sum(alpha_sol[l]*kernel(x_sol, x_sol, l, i, parametros) for l in 1:N)
    end

    # Imprimir el valor de la función objetivo
    println("valor", (x_gorro[N, 2]-vf)^2)
    println("velocidad final", x_gorro[N, 2])

    # Graficar las soluciones
    plotly()

    # Posición x
    plot1 = plot(0:T/N:T, x_gorro[:, 1], label="Posición x", xlabel="Tiempo", ylabel="Posición x", legend=:top, grid=true)

    # Velocidad v
    plot2 = plot(0:T/N:T, x_gorro[:, 2], label="Velocidad v", xlabel="Tiempo", ylabel="Velocidad v", legend=:top, grid=true, color=:orange)

    # Control
    plot3 = plot(0:T/N:T, u_opt, label="Control Óptimo", xlabel="Tiempo", ylabel="Control", legend=:top, grid=true)

    plot(plot1, plot2, plot3, layout=(3, 1))
end

plot(result, valores_iniciales, parametros)